In [70]:
!pip install langchain-ollama pymupdf PyPDF2


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for PyPDF2 from https://files.pythonhosted.org/packages/8e/5e/c86a5643653825d3c913719e788e41386bee415c2b87b4f955432f2de6b2/pypdf2-3.0.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   ----- ---------------------------------- 30.7/232.6 kB 1.3 MB/s eta 0:00:01
   -------------------------------------- - 225.3/232.6 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 232.6/232.6 kB 3.5 MB/s eta 0:00:00


In [71]:
import os
import pickle

from langchain_ollama import OllamaLLM  # Use updated OllamaLLM class
from langchain.document_loaders import PyMuPDFLoader
import PyPDF2
# from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [72]:
# Load Ollama model (ensure Ollama server is running locally)
ollama = OllamaLLM(base_url="http://localhost:11434", model="llama3.2")

In [94]:
# Define files and target years
files_to_predict = {
    "2018": ("./Tesla annual report 2018.pdf", "2019", "21.46 Million USD", "4.04 Million USD"),
    # "2020": ("./Walmart_2018_Annual_Report.pdf", "2021", "247.837 Billion USD", "4 Billion USD"),
    # "2022": ("./Walmart_2018_Annual_Report.pdf", "2023", "247.837 Billion USD", "4 Billion USD"),
}

chunk_cache_path = "./chunk_cache.pkl"  # File to store precomputed chunks

In [95]:
def extract_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

In [96]:
# Function to query Ollama for predictions
def query_ollama(ollama, prompt):
    response = ollama.invoke(input=prompt)
    return response

In [97]:
for year, (file_path, next_year, revenue, income) in files_to_predict.items():

    report_extract = extract_from_pdf(file_path)
    # Prediction template
    PREDICTION_TEMPLATE = f"""
    You are a financial analyst. Based on the following information from Tesla's {year} annual report and revenue numbers, predict the Revenues and Net Income for the year {next_year} 
    Please provide the predicted values for both Revenue and Net Income (or Net Sales). Be concise and provide reasoning for your prediction.

    {year} Revenue: {revenue},
    {year} net income: {income},

    Financial Highlights:
    {report_extract}
    """
    # print(PREDICTION_TEMPLATE)
    print(f'Using {year}\'s Report to predict {next_year}\'s Revenues - ')
    print(query_ollama(ollama, PREDICTION_TEMPLATE))


Using 2018's Report to predict 2019's Revenues - 
The document appears to be a certification statement for Tesla, Inc.'s Annual Report on Form 10-K for the annual period ended December 31, 2018. The certifications are made by two officers of the company:

1. **Elon Musk**, Chief Executive Officer (Principal Executive Officer)
2. **Deepak Ahuja**, Chief Financial Officer (Principal Financial Officer)

The statements certify that they have reviewed the report and have knowledge about its contents. They attest to the following:

* The report fully complies with the requirements of Section 13(a) or 15(d) of the Securities Exchange Act of 1934.
* The information contained in the report fairly presents, in all material respects, the financial condition and results of operations of Tesla, Inc.

The certifications are made under Section 1350 of Title 18, United States Code, which requires public companies to disclose certain material information about their financial statements. By signing thi